In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2021-07-30 13:11:50--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.5.128, 74.125.206.128, 64.233.184.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.5.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  57.1MB/s    in 1.1s    

2021-07-30 13:11:52 (57.1 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir,'train')
validation_dir = os.path.join(base_dir,'validation')

In [5]:
train_cats_dir = os.path.join(train_dir,'cats')
train_dogs_dir = os.path.join(train_dir,'dogs')

validation_cats_dir = os.path.join(validation_dir,'cats')
validation_dogs_dir = os.path.join(validation_dir,'dogs')

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [15]:
model1 = tf.keras.models.Sequential([
                tf.keras.layers.Flatten(input_shape = (150,150,3)),
                tf.keras.layers.Dense(256,activation = tf.nn.relu),
                tf.keras.layers.Dropout(0.5),
                tf.keras.layers.Dense(128,activation = tf.nn.relu),
                tf.keras.layers.Dense(1,activation = tf.nn.sigmoid)
])

In [16]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 67500)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               17280256  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 17,313,281
Trainable params: 17,313,281
Non-trainable params: 0
_________________________________________________________________


In [17]:
model1.compile(loss = 'binary_crossentropy',
               optimizer =RMSprop(1e-4),
               metrics = ['accuracy'])

In [18]:
history = model1.fit(
      train_generator,
      epochs = 30,
      validation_data = validation_generator,
      steps_per_epoch = 100,
      validation_steps = 50,
      verbose = 1
)

Epoch 1/30
100/100 [==============================] - 11s 101ms/step - loss: 7.4463 - accuracy: 0.5050 - val_loss: 0.6938 - val_accuracy: 0.4970
Epoch 2/30
100/100 [==============================] - 10s 102ms/step - loss: 0.8158 - accuracy: 0.5015 - val_loss: 0.6931 - val_accuracy: 0.4990
Epoch 3/30
100/100 [==============================] - 10s 98ms/step - loss: 0.7891 - accuracy: 0.5005 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/30
100/100 [==============================] - 10s 99ms/step - loss: 0.7292 - accuracy: 0.5020 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 5/30
100/100 [==============================] - 10s 100ms/step - loss: 0.7043 - accuracy: 0.4930 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/30
100/100 [==============================] - 10s 99ms/step - loss: 0.7724 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 7/30
100/100 [==============================] - 10s 98ms/step - loss: 0.7253 - accuracy: 0.4995 - val_loss: 0.6931 - val_accura

In [22]:
model2 = tf.keras.models.Sequential([
                    tf.keras.layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (150,150,3)),
                    tf.keras.layers.MaxPooling2D(2,2),
                    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
                    tf.keras.layers.MaxPooling2D(2,2),
                    tf.keras.layers.Conv2D(128,(3,3), activation = 'relu'),
                    tf.keras.layers.MaxPooling2D(2,2),
                    tf.keras.layers.Conv2D(128,(3,3), activation = 'relu'),
                    tf.keras.layers.MaxPooling2D(2,2),
                    tf.keras.layers.Dropout(0.5),
                    tf.keras.layers.Flatten(),
                    tf.keras.layers.Dense(512,activation = 'relu'),
                    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

In [23]:
model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 15, 15, 128)      

In [26]:
model2.compile(loss = 'binary_crossentropy',
               optimizer = RMSprop(learning_rate=1e-4),
               metrics = ['accuracy'])

In [28]:
history = model2.fit(
    train_generator,
    epochs = 20,
    validation_data = validation_generator,
    steps_per_epoch = 100,
    validation_steps = 50,
    verbose = 1
)

Epoch 1/20
100/100 [==============================] - 12s 115ms/step - loss: 0.6900 - accuracy: 0.5390 - val_loss: 0.6723 - val_accuracy: 0.5910
Epoch 2/20
100/100 [==============================] - 11s 112ms/step - loss: 0.6747 - accuracy: 0.5790 - val_loss: 0.6517 - val_accuracy: 0.6160
Epoch 3/20
100/100 [==============================] - 11s 112ms/step - loss: 0.6562 - accuracy: 0.5995 - val_loss: 0.6427 - val_accuracy: 0.6450
Epoch 4/20
100/100 [==============================] - 11s 112ms/step - loss: 0.6239 - accuracy: 0.6515 - val_loss: 0.6456 - val_accuracy: 0.6190
Epoch 5/20
100/100 [==============================] - 11s 111ms/step - loss: 0.5919 - accuracy: 0.6795 - val_loss: 0.6090 - val_accuracy: 0.6780
Epoch 6/20
100/100 [==============================] - 11s 111ms/step - loss: 0.5615 - accuracy: 0.7235 - val_loss: 0.5722 - val_accuracy: 0.7090
Epoch 7/20
100/100 [==============================] - 11s 111ms/step - loss: 0.5281 - accuracy: 0.7390 - val_loss: 0.6397 - val_ac